In [5]:
from shapely.geometry import Polygon
import numpy as np
from tqdm import tqdm
from rasterio.features import rasterize
import json


In [7]:
#CODE from solafune-tools
class PixelBasedMetrics:
    def __init__(self) -> None:
        pass
    def polygons_to_mask(self, polygons, array_dim):
        """
        Converts a list of polygons into a binary mask.
        
        Args:
            polygons (list): List of polygons, where each polygon is represented by a list of (x, y) tuples.
            array_dim (tuple): Dimensions of the output mask (height, width).
        
        Returns:
            np.ndarray: Binary mask with 1s for polygon areas and 0s elsewhere.
        """
        shapes = [(polygon, 1) for polygon in polygons]
        mask = rasterize(shapes, out_shape=array_dim, fill=0, dtype=np.uint8)
        return mask
    def compute_f1(self, gt_polygons, pred_polygons, array_dim=(1024, 1024)):
        """
        Compute the F1 score, precision, and recall for the given ground truth and predicted polygons.
        
        Args:
            gt_polygons (list): List of ground truth polygons.
            pred_polygons (list): List of predicted polygons.
            array_dim (tuple, optional): Dimensions of the output mask (height, width). Defaults to (1024, 1024).

        Returns:
            tuple: A tuple containing the F1 score, precision, and recall.
        """
        # Pixel-level improvement
        # Create binary masks for ground truth and predictions
        gt_mask = self.polygons_to_mask(gt_polygons, array_dim)
        pred_mask = self.polygons_to_mask(pred_polygons, array_dim)
        
        # Calculate pixel-level True Positives (TP), False Positives (FP), and False Negatives (FN)
        tp = np.sum((gt_mask == 1) & (pred_mask == 1))
        fp = np.sum((gt_mask == 0) & (pred_mask == 1))
        fn = np.sum((gt_mask == 1) & (pred_mask == 0))
        
        # Calculate precision, recall, and F1 score
        precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0  # if no prediction, precision is considered as 1
        recall = tp / (tp + fn) if (tp + fn) > 0 else 1.0  # if no ground truth, recall is considered as 1
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0  # if either precision or recall is 0, f1 is 0
        
        return f1, precision, recall

In [ ]:
#Function which return the total F1 score for the training data
def f1_scorer(path_ground, path_pred):
    with open(f"{path_ground}.json", 'r') as file:
        train_annotations = json.load(file)
    with open(f"{path_pred}.json", 'r') as file:
        pred_annotations = json.load(file)
    score=[]
    for i in range(len(train_annotations["images"])):
        gt_polygons = {'plantation' : [],
        'grassland_shrubland': [],
        'mining' :[],
        'logging':[]}
        pred_polygons = {'plantation' : [],
        'grassland_shrubland': [],
        'mining' :[],
        'logging':[]}
        for j in range(len(train_annotations["images"][i]["annotations"])):
            unproccesed_poly=train_annotations["images"][i]["annotations"][j]["segmentation"]
            processed_poly= Polygon(list(zip(unproccesed_poly[::2], unproccesed_poly[1::2])))
            gt_polygons[train_annotations["images"][i]["annotations"][j]["class"]].append(processed_poly)

        for j in range(len(pred_annotations["images"][i]["annotations"])):
            unproccesed_poly=pred_annotations["images"][i]["annotations"][j]["segmentation"]
            processed_poly= Polygon(list(zip(unproccesed_poly[::2], unproccesed_poly[1::2])))
            pred_polygons[pred_annotations["images"][i]["annotations"][j]["class"]].append(processed_poly)
        for key in gt_polygons.keys():

            if len(gt_polygons[key]) == 0 and len(pred_polygons[key]) == 0:
                score.append(1)
                continue
            if len(gt_polygons[key]) == 0 or len(pred_polygons[key]) == 0:
                score.append(0)
                continue
            f1, precision, recall = PixelBasedMetrics().compute_f1(gt_polygons[key], pred_polygons[key])
            score.append(f1)
        
    return sum(score)/len(score)
print(f1_scorer("train_annotations","train_annotations"))


{'plantation': [<POLYGON ((0 449, 9 454, 18 461, 26 468, 33 475, 40 477, 50 485, 59 497, 70 ...>, <POLYGON ((294 0, 299 11, 308 26, 326 34, 349 35, 371 36, 394 41, 415 47, 42...>, <POLYGON ((682 859, 683 880, 698 909, 723 949, 726 953, 730 976, 734 982, 74...>], 'grassland_shrubland': [], 'mining': [], 'logging': []}
{'plantation': [<POLYGON ((150 337, 139 347, 174 365, 167 375, 176 383, 184 373, 201 388, 20...>, <POLYGON ((216 235, 204 244, 211 255, 213 264, 207 267, 208 271, 210 279, 21...>, <POLYGON ((0 133, 10 121, 13 124, 11 130, 16 131, 23 125, 32 127, 29 131, 26...>, <POLYGON ((144 757, 153 742, 160 740, 162 748, 163 748, 176 752, 173 764, 16...>, <POLYGON ((242 256, 263 234, 283 223, 304 223, 291 235, 283 245, 287 271, 28...>], 'grassland_shrubland': [<POLYGON ((0 187, 8 188, 8 201, 0 202, 0 187))>, <POLYGON ((340 85, 353 77, 354 80, 361 78, 367 89, 353 103, 340 85))>, <POLYGON ((302 99, 305 98, 308 93, 310 94, 311 97, 318 98, 321 100, 319 104,...>, <POLYGON ((305 156, 304 159,